In [17]:
import pandas as pd
import numpy as np
import sklearn.cluster as sc
import sklearn.metrics as smt
from scipy.cluster import hierarchy
from collections import defaultdict
import pickle
import sklearn
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

# Plotting config
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#pip install pymorphy2
#pip install -U pymorphy2-dicts-ru
import pymorphy2

In [3]:
from sklearn.base import BaseEstimator, ClusterMixin


class Clustering(BaseEstimator, ClusterMixin):
    """
    Implement clustering algorithm according 
    """
    
    def __init__(self, metric='euclidean', n_clusters=1,linkage='average',distance_threshold=None):
        """
        metric - string with name of metric, for example, euclidean
        Please add necessary algoritm parameters to class constructor.
        """
        self.n_clusters=n_clusters
        self.distance_threshold=distance_threshold
        if distance_threshold is not None and n_clusters is not None:
            print('If distance_threshold is not None - n_clusters shoud be None')
            raise
            
        self.dist=None
        self.clusters=None
        self.ans=None
        self.Z=None
        
        
        if metric=='euclidean':
            self.metric=self.__eucl
        elif metric=='manhattan':
            self.metric=self.__manh
        elif metric=='cosine':
            self.metric=self.__cosine
        elif metric=='precomputed':
            self.metric=lambda x:x
        else:
            print('invalid metric')
            raise
        
        if linkage=='single':
            self.link=self.__single
        elif linkage=='complete':
            self.link=self.__compl
        elif linkage=='average':
            self.link=self.__aver
        else:
            print('invalid linkage')
            raise
    
    def __eucl(self,X):
        return np.sqrt(((X[...,np.newaxis]-X.T[np.newaxis])**2).sum(axis=1))
    
    def __manh(self,X):
        return (np.abs(X[...,np.newaxis]-X.T[np.newaxis])).sum(axis=1)    
    
    def __cosine(self,X):
        tmp=(X[...,np.newaxis]*X.T[np.newaxis]).sum(axis=1)
        norm=np.sqrt(tmp.diagonal())
        mask=norm==0
        norm[mask]=1
        return 1-tmp/norm[np.newaxis]/norm[...,np.newaxis]
    
    def __single(self,u,v):
        mask=self.dist[u]>self.dist[v]
        
        self.dist[u,mask]=self.dist[v,mask]
        self.dist[:,u]=self.dist[u]

        self.dist[v]=np.inf
        self.dist[:,v]=np.inf
        
        self.dist[u,u]=np.inf
            
    def __compl(self,u,v):
        mask=self.dist[u]<self.dist[v]
        
        self.dist[u][mask]=self.dist[v][mask]
        self.dist[:,u]=self.dist[u]
        
        self.dist[v]=np.inf
        self.dist[:,v]=np.inf
        
        self.dist[u,u]=np.inf
    
    def __aver(self,u,v):
        ai=len(self.clusters[u])
        aj=len(self.clusters[v])
        N=ai+aj
        ai/=N
        aj/=N
        
        self.dist[u]=ai*self.dist[u]+aj*self.dist[v]
        self.dist[:,u]=self.dist[u]
        
        self.dist[v]=np.inf
        self.dist[:,v]=np.inf
        
        self.dist[u,u]=np.inf
    
    def fit(self,x):
        self.dist=self.metric(x)
        n_clusters=N=cur_cluster=len(x)
        self.ans=np.arange(N)
        
        self.clusters=dict()
        for i in range(n_clusters):
            self.clusters[i]=[i]
            self.dist[i,i]=np.inf
        
        if self.n_clusters==1:
            self.Z=np.empty((N-1,4))
            cur_z=0
        
        while(n_clusters!=self.n_clusters):
            pos=self.dist.argmin()
            distanse=self.dist.ravel()[pos]
            if self.distance_threshold is not None:
                if distanse>self.distance_threshold:
                    break
            u=pos//N
            v=pos%N
            
            max_cluster=u if len(self.clusters[u])>=len(self.clusters[v]) else v
            min_cluster=u if max_cluster==v else v
            u,v=max_cluster,min_cluster
            
            if self.n_clusters==1:
                self.Z[cur_z]=[self.ans[self.clusters[u][0]],self.ans[self.clusters[v][0]],
                               max(0,distanse),len(self.clusters[u])+len(self.clusters[v])]
                cur_z+=1
                
            self.link(u,v)
            
            for i in self.clusters[v]:
                self.clusters[u].append(i)

            self.ans[self.clusters[u]]=cur_cluster

            cur_cluster+=1
            
            n_clusters-=1
            if(n_clusters==1):
                break
        return self
    
    def predict(self):
        return self.ans
        
    def fit_predict(self, x):
        """
        Use data matrix x to compute model parameters and predict clusters
        """
        self.fit(x)
        return self.predict()
    
    def plot_dendrogram(self):
        """
        Try to visualize our data
        """
        if self.Z is not None:
            hierarchy.dendrogram(self.Z)
            plt.show()
        else:
            print('Tree must be full')
        

In [4]:
def batch_generator(X, y, shuffle=True, batch_size=1):
    """
    Гератор новых батчей для обучения
    X          - матрица объекты-признаки
    y_batch    - вектор ответов
    shuffle    - нужно ли случайно перемешивать выборку
    batch_size - размер батча ( 1 это SGD, > 1 mini-batch GD)
    Генерирует подвыборку для итерации спуска (X_batch, y_batch)
    """
    
    idx=np.arange(y.shape[0])
    if shuffle:
        np.random.shuffle(idx)
    for i in range(0,y.shape[0],batch_size):
        X_batch = X[idx[i:min(i+batch_size,X.shape[0])]]
        y_batch = y[idx[i:min(i+batch_size,X.shape[0])]]
        yield (X_batch, y_batch)

# Теперь можно сделать генератор по данным ()
#  my_batch_generator = batch_generator(X, y, shuffle=True, batch_size=1):

In [5]:
#%%pycodestyle

def sigmoid(x):
    """
    Вычисляем значение сигмоида.
    X - выход линейной модели
    """
    
    ## Your code Here
    e=np.exp(x)
    sigm_value_x=e/(1+e)
    sigm_value_x[sigm_value_x==0]=0.0000000000000001
    sigm_value_x[sigm_value_x==1]=0.9999999999999999
    return sigm_value_x


from sklearn.base import BaseEstimator, ClassifierMixin

class MySGDClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, batch_generator, C=1, alpha=0.01, max_epoch=10, model_type='lin_reg'):
        """
        batch_generator -- функция генератор, которой будем создавать батчи
        C - коэф. регуляризации
        alpha - скорость спуска
        max_epoch - максимальное количество эпох
        model_type - тим модели, lin_reg или log_reg
        """
        
        self.C = C
        self.alpha = alpha
        self.max_epoch = max_epoch
        self.batch_generator = batch_generator
        self.errors_log = {'iter' : [], 'loss' : []}  
        self.model_type = model_type
        
    def calc_loss(self, X_batch, y_batch):
        """
        Считаем функцию потерь по батчу 
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """
        if self.model_type=='lin_reg':
            loss=((y_batch-self.predict(X_batch))**2).sum()/X_batch.shape[0]
        else:
            loss=-(np.dot(y_batch[np.newaxis],np.log2(self.predict(X_batch))[...,np.newaxis])\
                    +np.dot((1-y_batch)[np.newaxis],np.log2(1-self.predict(X_batch))[...,np.newaxis]))[0,0]\
                    /X_batch.shape[0]
            
        return loss
    
    def calc_loss_grad(self, X_batch, y_batch):
        """
        Считаем  градиент функции потерь по батчу (то что Вы вывели в задании 1)
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """

        loss_grad=np.dot((self.predict(X_batch)-y_batch)[np.newaxis],\
                         np.hstack((np.ones((X_batch.shape[0],1)),X_batch))).ravel()/X_batch.shape[0]
        if self.model_type=='lin_reg':
            loss_grad*=2
        else:
            from math import log
            loss_grad/=log(2)
            
        loss_grad[1:]+=2*self.weights[1:]/self.C
        
        return loss_grad
    
    def update_weights(self, new_grad):
        """
        Обновляем вектор весов
        new_grad - градиент по батчу
        """
        self.weights-=self.alpha*new_grad
    
    def fit(self, X, y,batch_size=1):
        '''
        Обучение модели
        X - матрица объекты-признаки
        y - вектор ответов
        '''
        
        # Нужно инициализровать случайно веса
        np.random.seed(0)
        self.weights = np.random.ranf(size=X.shape[1]+1).astype(np.float64)
        for n in range(0, self.max_epoch):
            new_epoch_generator = self.batch_generator(X,y,batch_size=batch_size)
            for batch_num, new_batch in enumerate(new_epoch_generator):
                X_batch = new_batch[0]
                y_batch = new_batch[1]
                batch_grad = self.calc_loss_grad(X_batch, y_batch)
                self.update_weights(batch_grad)
                # Подумайте в каком месте стоит посчитать ошибку для отладки модели
                # До градиентного шага или после
                batch_loss = self.calc_loss(X_batch, y_batch)
                self.errors_log['iter'].append(batch_num)
                self.errors_log['loss'].append(batch_loss)
                
        return self
        
    def predict(self, X):
        '''
        Предсказание класса
        X - матрица объекты-признаки
        Не забудте тип модели (линейная или логистическая регрессия)!
        '''
        X=X.reshape(-1,self.weights.shape[0]-1)
        if self.model_type=='lin_reg':
            y_hat=np.dot(X,self.weights[1:][...,np.newaxis]).ravel()+self.weights[0]
        else:
            y_hat=sigmoid(np.dot(X,self.weights[1:][...,np.newaxis]).ravel()+self.weights[0])
            
        # Желательно здесь использовать матричные операции между X и весами, например, numpy.dot 
        return y_hat

In [6]:
doc_to_title = {}
with open('docs_titles.tsv',encoding="utf8") as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print(len(doc_to_title))

28026


**Удаляю лишнее и разбиваю предложения на слова**

In [7]:
delete_words={'на','в','и','или','от','для','про','под','об','над','за','у','с','через','при','перед','от','о','по','на','к',
            'из','до','без'}

for key,line in doc_to_title.items():
    line=line.lower()
    tmp=list(line)
    for i, c in enumerate(line):
        if(not(c.isalpha())):
            tmp[i]=' '
    tmp=list(filter(lambda x: x not in delete_words and len(x)>1,''.join(tmp).split()))
    doc_to_title[key]=tmp

**Нормализую слова**

In [8]:
morph = pymorphy2.MorphAnalyzer()
for key,lst in doc_to_title.items():
    doc_to_title[key]=[morph.parse(x)[0].normal_form for x in lst]

In [11]:
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [14]:
link='single'
y_true=[]
Y_pred1=[]

for gr,value in traingroups_titledata.items():
    words=dict()
    for i in value:
        for word in i[1]:
            if word not in words:
                words[word]=len(words)

    x=np.zeros((len(value),len(words)+1))
    y=np.empty(len(value)).astype(int)
    
    
    d=defaultdict(int)
    for ind,i in enumerate(value):
        
        for word in i[1]:
            x[ind,words[word]]+=1
            if i[2]:
                d[word]+=1
        y[ind]=i[2]
        y_true.append(i[2])
        
   
    
    old_x=x
    s=x.sum(axis=1)<=1
    x[s]=0
    
    сustum_aggl = Clustering(metric='cosine',n_clusters=None,linkage=link,distance_threshold=0.55)
    y_pred=сustum_aggl.fit_predict(x)
    
    tmp=y_pred
    
    un=np.unique(y_pred)
    count=(y_pred[...,np.newaxis]==un[np.newaxis]).sum(axis=0)
    mask=((y_pred[...,np.newaxis]==un[count<=4][np.newaxis]).sum(axis=1)).astype(bool)

    
    y_pred[mask]=0
    y_pred[~mask]=1
    Y_pred1+=list(y_pred)
    
print('fscore =',smt.f1_score(y_true,Y_pred1))



fscore = 0.7197093096103245


In [18]:
y_true=[]
Y_pred2=[]

for gr,value in traingroups_titledata.items():
    words=dict()
    for i in value:
        for word in i[1]:
            if word not in words:
                words[word]=len(words)

    x=np.zeros((len(value),len(words)+1))
    y=np.empty(len(value)).astype(int)
    
    
    d=defaultdict(int)
    for ind,i in enumerate(value):
        
        for word in i[1]:
            x[ind,words[word]]=1
            if i[2]:
                d[word]+=1
        y[ind]=i[2]
        y_true.append(i[2])
    
    s=x.sum(axis=1)<=1
    x[s]=0
    
    old_x=x
    
    inter=np.dot(x,x.T)
    s=x.sum(axis=1)
    union=s[...,np.newaxis]+s[np.newaxis]
    union-=inter
    union[union==0]=1
    x=inter/union
    x=1-x
    
    model=sklearn.cluster.DBSCAN(eps=0.7,min_samples=5,metric='precomputed',algorithm='brute').fit(x)
    y_pred=model.labels_

    tmp=y_pred
    
   
    
    mask=y_pred==-1
    
    y_pred[mask]=0
    y_pred[~mask]=1
    Y_pred2+=list(y_pred)

print('fscore =',smt.f1_score(y_true,Y_pred2))


fscore = 0.7187928669410151


In [19]:
y_train = []
X_train = []
groups_train = []
N1=5
N2=5
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    value=docs
    
    words=dict()
    for i in value:
        y_train.append(i[2])
        for word in i[1]:
            if word not in words:
                words[word]=len(words)
            
    x=np.zeros((len(value),len(words)+1))

    for ind,i in enumerate(value):
        for word in i[1]:
            x[ind,words[word]]+=1
            
    
    count_x=x
    bool_x=(x>=1).astype(int)
    
    s=bool_x.sum(axis=1)<=1
    
    bool_x[s]=0
    count_x[s]=0
    
    inter=np.dot(bool_x,bool_x.T)
    s=bool_x.sum(axis=1)
    union=s[...,np.newaxis]+s[np.newaxis]
    union-=inter
    union[union==0]=1
    x=inter/union
    x=1-x
    
    
    l=[(0.01,5),(0.05,5),(0.1,5),(0.01,4),(0.05,4),
       (0.1,4),(0.2,5),(0.25,5),(0.2,4),(0.25,4),(0.4,5),(0.4,4),(0.65,5),(0.75,5),(0.85,5),(1,1)]
    
    for (eps,min_s) in l:
        model=sklearn.cluster.DBSCAN(eps=eps,min_samples=min_s,metric='precomputed',algorithm='brute').fit(x)
        y_pred=model.labels_
        mask=y_pred==-1
        if y_pred[~mask].shape[0]>=N1:
            break
    
    if (eps,min_s)==l[-1]:
        x=np.zeros((len(value),N1)) 
    else:
        target_x =bool_x[~mask]
        inter=np.dot(bool_x,target_x.T)
        union=bool_x.sum(axis=1)[...,np.newaxis]+target_x.sum(axis=1)[np.newaxis]
        union-=inter
        union[union==0]=1
        x=inter/union
        
        sort_ind=np.argsort(x,axis=1)
        x=np.take_along_axis(x, sort_ind, axis=1)[:,::-1]
        x=x[:,:N1]
    
    for k, (doc_id, title, target_id) in enumerate(docs):
        all_dist = []

        title_words = set(title)
        words=title_words
        for j in range(0, len(docs)):
            if k == j:
                continue

            _, title_j, _ = docs[j]
            title_words_j = set(title_j)

            words_j=title_words_j
            if len(words)==1:
                all_dist.append(0)
            else:
                inter=len(words.intersection(words_j))
                all_dist.append(0 if inter==0 else inter/(len(words)+len(words_j)-inter))


        tmp=sorted(all_dist, reverse=True)[0:N2]
        tmp+=list(x[k])
        X_train.append(tmp)
        
        
        
    for i in range(len(x)):
        groups_train.append(new_group)
        
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 10) (11690,) (11690,)


In [20]:
X_train=np.hstack([X_train,np.array(Y_pred1)[...,np.newaxis]])

In [21]:
S_all=[]
for value in np.unique(groups_train):
    S_all.append(groups_train==value)

In [22]:
scaler = StandardScaler()
scaler.fit(X_train)
X=scaler.transform(X_train)

In [23]:
minlos=1e10
for alpha in [0.1,0.05,0.01,0.2,0.3]:
    for c in [2*alpha/0.1,2*alpha/0.05]:
        for model_type in ['log_reg']:
            for max_epoch in [50,100,150,200]:
                loss=0
                for s in S_all[:10]:
                    model=MySGDClassifier(batch_generator,alpha=alpha,C=c,model_type=model_type,max_epoch=max_epoch)
                    model.fit(X[~s],y_train[~s],batch_size=11690)
                    loss+=model.calc_loss(X[s],y_train[s])
                if loss<minlos:
                    minlos=loss
                    best_alpha=alpha
                    best_c=c
                    best_max_epoch=max_epoch
                    best_model=model

In [24]:
print(f'best_alpha={best_alpha} best_c={best_c} best_epoch={best_max_epoch}')

best_alpha=0.1 best_c=4.0 best_epoch=50


In [25]:
best_max_epoch=100

In [26]:
best_alpha=0.1
best_c=4
best_max_epoch=50

In [27]:
model=MySGDClassifier(batch_generator,alpha=best_alpha,C=best_c,model_type='log_reg',max_epoch=best_max_epoch)
model.fit(X,y_train,batch_size=11690)
best_f=0
for th in np.linspace(0.1,1,1000,endpoint=False):
    y=model.predict(X)>=th
    f=smt.f1_score(y_train,y)
    if f>best_f:
        best_f=f
        best_th=th
best_model=model

In [28]:
print(f'best_f={best_f} best_th={best_th}')

best_f=0.726380042462845 best_th=0.40869999999999995


In [29]:
y_true=np.zeros((0,1))
for s in S_all:
    y_true=np.vstack([y_true,y_train[s][...,np.newaxis]])

In [30]:
f=0
models=[]
y_pred=np.zeros((0,1))
for s in S_all:
    model=MySGDClassifier(batch_generator,alpha=best_alpha,C=best_c,model_type='log_reg',max_epoch=best_max_epoch)
    model.fit(X[~s],y_train[~s],batch_size=11690)
    y=model.predict(X[s])>=best_th
    y_pred=np.vstack([y_pred,y[...,np.newaxis]])
    models.append(model)
    
print(smt.f1_score(y_true,y_pred))

0.7247645576336385


In [31]:
test_data = pd.read_csv('test_groups.csv')
testgroups_titledata = {}
pair_id=[]
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))
    pair_id.append(new_doc['pair_id'])
pair_id=np.array(pair_id)

In [32]:
link='single'

Y_pred1_test=[]


for gr,value in testgroups_titledata.items():
    words=dict()
    for i in value:
        for word in i[1]:
            if word not in words:
                words[word]=len(words)

    x=np.zeros((len(value),len(words)+1))
    
    
   
    for ind,i in enumerate(value):
        
        for word in i[1]:
            x[ind,words[word]]=1
            
    s=x.sum(axis=1)<=1
    x[s]=0
    
    сustum_aggl = Clustering(metric='cosine',n_clusters=None,linkage=link,distance_threshold=0.55)
    y_pred=сustum_aggl.fit_predict(x)
    
   
    un=np.unique(y_pred)
    count=(y_pred[...,np.newaxis]==un[np.newaxis]).sum(axis=0)
    mask=((y_pred[...,np.newaxis]==un[count<=4][np.newaxis]).sum(axis=1)).astype(bool)
  
    
    y_pred[mask]=0
    y_pred[~mask]=1
    Y_pred1_test+=list(y_pred)
Y_pred1_test=np.array(Y_pred1_test)
print(Y_pred1_test.shape)

(16627,)


In [33]:
X_test = []
groups_test = []
N1=5
N2=5
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    value=docs
    
    words=dict()
    for i in value:
        for word in i[1]:
            if word not in words:
                words[word]=len(words)

    x=np.zeros((len(value),len(words)+1))

    for ind,i in enumerate(value):
        for word in i[1]:
            x[ind,words[word]]+=1
            
    
    count_x=x
    bool_x=(x>=1).astype(int)
    
    s=bool_x.sum(axis=1)<=1
    
    bool_x[s]=0
    count_x[s]=0
    
    inter=np.dot(bool_x,bool_x.T)
    s=bool_x.sum(axis=1)
    union=s[...,np.newaxis]+s[np.newaxis]
    union-=inter
    union[union==0]=1
    x=inter/union
    x=1-x
    
    l=[(0.01,5),(0.05,5),(0.1,5),(0.01,4),(0.05,4),
       (0.1,4),(0.2,5),(0.25,5),(0.2,4),(0.25,4),(0.4,5),(0.4,4),(0.65,5),(0.75,5),(0.85,5),(1,1)]
    
    for (eps,min_s) in l:
        model=sklearn.cluster.DBSCAN(eps=eps,min_samples=min_s,metric='precomputed',algorithm='brute').fit(x)
        y_pred=model.labels_
        mask=y_pred==-1
        if y_pred[~mask].shape[0]>=N1:
            break
    
    if (eps,min_s)==l[-1]:
        x=np.zeros((len(value),N1))
    else:
        target_x =bool_x[~mask]
        inter=np.dot(bool_x,target_x.T)
        union=bool_x.sum(axis=1)[...,np.newaxis]+target_x.sum(axis=1)[np.newaxis]
        union-=inter
        union[union==0]=1
        x=inter/union
        
        sort_ind=np.argsort(x,axis=1)
        x=np.take_along_axis(x, sort_ind, axis=1)[:,::-1]
        x=x[:,:N1]
    
    for k, (doc_id, title) in enumerate(docs):
        all_dist = []

        title_words = set(title)
        words=title_words
        for j in range(0, len(docs)):
            if k == j:
                continue

            _, title_j = docs[j]
            title_words_j = set(title_j)
            
            words_j=title_words_j
            if len(words)==1:
                all_dist.append(0)
            else:
                inter=len(words.intersection(words_j))
                all_dist.append(0 if inter==0 else inter/(len(words)+len(words_j)-inter))

        tmp=sorted(all_dist, reverse=True)[0:N2]
        tmp+=list(x[k])
        X_test.append(tmp)
        
        
        
    for i in range(len(x)):
        groups_test.append(new_group)
        
X_test = np.array(X_test)
groups_test = np.array(groups_test)
print (X_test.shape, groups_test.shape)

(16627, 10) (16627,)


In [34]:
X_test=np.hstack([X_test,np.array(Y_pred1_test)[...,np.newaxis]])
X_test=scaler.transform(X_test)

In [35]:
model=best_model
y_test=model.predict(X_test)>=best_th
print(y_test.shape)

(16627,)


In [36]:
with open('ans.csv','w',encoding="utf8") as f:
    print('pair_id,target',file=f)
    for i,pair in enumerate(pair_id):
        print(pair,f',{y_test[i]}',file=f,sep='')

**Финальный fscore = 0.71887**